In [1]:
import findspark

import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import histogram_numeric, col, lit
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import gc

import os
import sys

findspark.init()

# os.environ['PYSPARK_PYTHON'] = sys.executable
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
spark = SparkSession.builder.appName("Insurance Cross Selling").getOrCreate()
spark

In [3]:
train_df = spark.read.format("csv").options(
        header=True,
        inferSchema=True
    ).load("data/train.csv")
train_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Driving_License: integer (nullable = true)
 |-- Region_Code: double (nullable = true)
 |-- Previously_Insured: integer (nullable = true)
 |-- Vehicle_Age: string (nullable = true)
 |-- Vehicle_Damage: string (nullable = true)
 |-- Annual_Premium: double (nullable = true)
 |-- Policy_Sales_Channel: double (nullable = true)
 |-- Vintage: integer (nullable = true)
 |-- Response: integer (nullable = true)



In [4]:
majority_df = train_df.filter(col("Response") == 0)
minority_df = train_df.filter(col("Response") == 1)

ratio = int(majority_df.count() / minority_df.count())

undersampled_majority = majority_df.sample(withReplacement=False, fraction=1 / ratio)
resampled_df = undersampled_majority.unionAll(minority_df)

In [5]:
# categorical_cols = [
#     "Gender",
#     "Driving_License",
#     "Region_Code",
#     "Previously_Insured",
#     "Vehicle_Age",
#     "Vehicle_Damage",
#     "Policy_Sales_Channel",
#     "Response",
# ]

string_categorical_in_cols = [
    "Gender",
    "Vehicle_Age",
    "Vehicle_Damage",
]
string_categorical_out_cols = [f"{s}_ind" for s in string_categorical_in_cols]

features_cols = [
    f for f in train_df.columns if f not in ["Response", "id"] + string_categorical_in_cols
] + string_categorical_out_cols
pred_col = "Response"

string_indexers = [
    StringIndexer(inputCol=in_col, outputCol=out_col)
    for in_col, out_col in zip(string_categorical_in_cols, string_categorical_out_cols)
]

assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
rf = RandomForestClassifier(labelCol=pred_col, featuresCol="features", seed=1)

pipeline = Pipeline(stages=string_indexers + [assembler, rf])

evaluator = BinaryClassificationEvaluator(labelCol=pred_col)
# rf_paramgrid = (
#     ParamGridBuilder()
#     .addGrid(rf.numTrees, list([i for i in range(10, 30, 10)]))
#     .build()
# )

# cv = CrossValidator(
#     estimator=pipeline, estimatorParamMaps=rf_paramgrid, evaluator=evaluator, numFolds=2
# )


In [6]:
model = pipeline.fit(train_df)

In [7]:
model.write().overwrite().save(f"{os.path.join(os.getcwd(), 'sparkmodels', 'rf')}")

Py4JJavaError: An error occurred while calling o370.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java